<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec
import os
import glob, pickle, time
import re
import unicodedata

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### Datos
Utilizaremos como dataset el libro los juegos del hambre

In [3]:
import os

# Ruta de la carpeta que contiene los libros
dataset_path = '/content/drive/My Drive/Colab Notebooks/hungergames.txt'

# Lista para almacenar los capítulos
corpus = []

# Abro el archivo y lee las líneas
with open(dataset_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Inicializo una variable para el capítulo actual
current_chapter = ''

# Recorro las líneas del archivo
for line in lines:
    # la línea contiene el nombre del capítulo entre guiones bajos
    if re.match(r'_+\s+\d+\s+_+', line):
        # Agrego el capítulo actual al corpus si no está vacío
        if current_chapter:
            corpus.append(current_chapter.strip())
        # Inicializo el capítulo actual con la línea actual
        current_chapter = line
    else:
        # Agrego la línea al capítulo actual
        current_chapter += line

# Añado el último capítulo al corpus
if current_chapter:
    corpus.append(current_chapter.strip())

# Imprimo el número de capítulos y el corpus
print(f'Número de capítulos: {len(corpus)}')



Número de capítulos: 28


In [5]:
# Inicializa una lista para almacenar las oraciones
sentences = []

# Divido cada capítulo en oraciones
for text in corpus:
    chapter_sentences = text.split('.')
    chapter_sentences = [sentence.strip() for sentence in chapter_sentences if sentence.strip()]
    sentences.extend(chapter_sentences)

df = pd.DataFrame({'Oraciones': sentences})
print("Cantidad de oraciones:", df.shape[0])



Cantidad de oraciones: 5960


### 1 - Preprocesamiento

In [6]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
# Demos un vistazo
sentence_tokens[:2]

[['los', 'juegos', 'del', 'hambre', 'saga', 'distritos', 'vol'],
 ['1',
  'suzanne',
  'collins',
  '©',
  '2008',
  'the',
  'hunger',
  'games',
  'traducción',
  'pilar',
  'ramírez',
  'tello',
  'primera',
  'parte',
  'los',
  'tributos']]

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=10,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 5960


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 1087


### 3 - Entrenar embeddings

In [13]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=30,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 637908.6875
Loss after epoch 1: 515134.0625
Loss after epoch 2: 457866.125
Loss after epoch 3: 454233.75
Loss after epoch 4: 390123.875
Loss after epoch 5: 384437.25
Loss after epoch 6: 383309.75
Loss after epoch 7: 382378.25
Loss after epoch 8: 379159.25
Loss after epoch 9: 366769.5
Loss after epoch 10: 353877.5
Loss after epoch 11: 351512.5
Loss after epoch 12: 348936.0
Loss after epoch 13: 347521.5
Loss after epoch 14: 345057.0
Loss after epoch 15: 343871.5
Loss after epoch 16: 341785.0
Loss after epoch 17: 339720.5
Loss after epoch 18: 339970.5
Loss after epoch 19: 337894.0
Loss after epoch 20: 335762.5
Loss after epoch 21: 331018.0
Loss after epoch 22: 320677.0
Loss after epoch 23: 317040.0
Loss after epoch 24: 317809.0
Loss after epoch 25: 316242.0
Loss after epoch 26: 315318.0
Loss after epoch 27: 314164.0
Loss after epoch 28: 313424.0
Loss after epoch 29: 312494.0


(1523519, 3001050)

### 4 - Ensayar

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["tributo"], topn=10)

[('3', 0.5885858535766602),
 ('trabajo', 0.5846183896064758),
 ('mentor', 0.5808099508285522),
 ('importante', 0.5627768039703369),
 ('viejo', 0.5624746680259705),
 ('vencedor', 0.5612919926643372),
 ('juego', 0.5591087937355042),
 ('carro', 0.5576795935630798),
 ('12', 0.5543392300605774),
 ('haberse', 0.540571928024292)]

In [41]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["hambre"], topn=10)

[('quieren', 0.4632849395275116),
 ('muerta', 0.45178642868995667),
 ('reacción', 0.43618372082710266),
 ('tus', 0.43526989221572876),
 ('¡no', 0.4338763952255249),
 ('vencedores', 0.42313113808631897),
 ('estos', 0.41975682973861694),
 ('vivos', 0.4159022569656372),
 ('creía', 0.41153037548065186),
 ('año', 0.40934479236602783)]

In [48]:
w2v_model.wv.most_similar(positive=["peeta"], topn=10)

[('¡peeta', 0.5389680862426758),
 ('asiento', 0.49447181820869446),
 ('¿dónde', 0.49078357219696045),
 ('madge', 0.47503069043159485),
 ('oh', 0.4642370939254761),
 ('rabia', 0.44612911343574524),
 ('sienta', 0.44378402829170227),
 ('¿lo', 0.4376392960548401),
 ('duda', 0.43658649921417236),
 ('solo', 0.4334806203842163)]

In [15]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["peeta"], topn=10)

[('hay', -0.02148137241601944),
 ('llegar', -0.04958915710449219),
 ('estadio', -0.053203269839286804),
 ('ciudad', -0.06511428952217102),
 ('son', -0.06651492416858673),
 ('horas', -0.07456271350383759),
 ('cuantas', -0.0791088119149208),
 ('cuatro', -0.08140630275011063),
 ('hojas', -0.0841616615653038),
 ('campo', -0.08501248061656952)]

In [16]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["distrito"], topn=10)

[('2', 0.6834509968757629),
 ('3', 0.6495717167854309),
 ('11', 0.6151982545852661),
 ('4', 0.6035338044166565),
 ('12', 0.5962912440299988),
 ('1', 0.5893728733062744),
 ('carro', 0.5748105645179749),
 ('quedarse', 0.5603776574134827),
 ('cadáver', 0.557036280632019),
 ('pareja', 0.5547200441360474)]

In [31]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["arco"], topn=10)

[('carcaj', 0.7306239604949951),
 ('cinturón', 0.5926663875579834),
 ('flechas', 0.5925911664962769),
 ('cuchillo', 0.5764560699462891),
 ('posición', 0.5206767320632935),
 ('juego', 0.5116968154907227),
 ('lanzo', 0.5080978274345398),
 ('cojo', 0.5058712363243103),
 ('hombro', 0.5021395683288574),
 ('hoja', 0.5017749667167664)]

In [36]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["sobrevivir"], topn=10)

[('matar', 0.6614676117897034),
 ('haberlo', 0.6133893728256226),
 ('empezar', 0.6106350421905518),
 ('patrocinadores', 0.6092326045036316),
 ('conseguir', 0.6075491309165955),
 ('vivir', 0.5747507810592651),
 ('éxito', 0.5747399926185608),
 ('ello', 0.5731070637702942),
 ('entrar', 0.5646550059318542),
 ('darle', 0.5596312880516052)]

In [30]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["ganar"], topn=10)

[('vencedores', 0.6508707404136658),
 ('vivir', 0.6208860278129578),
 ('juntos', 0.6141685247421265),
 ('tuviese', 0.6121072769165039),
 ('tendrán', 0.609208345413208),
 ('pareja', 0.5931529998779297),
 ('reglas', 0.5916980504989624),
 ('patrocinadores', 0.5871109962463379),
 ('seguimos', 0.5845211148262024),
 ('viendo', 0.5795626640319824)]

In [19]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("arco")
print(vector_love)

[ 5.29887199e-01  3.27084899e-01 -3.21644880e-02 -2.23858595e-01
  5.18327318e-02 -9.26573016e-03  3.62304807e-01  1.30542830e-01
  2.90606096e-02  7.10215941e-02  1.36427149e-01 -1.56318590e-01
 -2.89924622e-01  5.26733279e-01 -8.44235420e-02 -8.27237591e-02
  1.12002261e-01  2.06472158e-01  7.77824074e-02  1.27979973e-02
  8.77769962e-02 -1.52230442e-01  3.29303704e-02  2.24135369e-01
 -7.83382952e-02 -8.92772898e-03 -2.13668942e-01  1.49978464e-02
 -1.59502357e-01 -2.21605241e-01  1.44510180e-01 -1.14951290e-01
 -9.32073146e-02  7.81046972e-02 -1.62562415e-01  4.83121067e-01
  4.41701770e-01 -3.03260148e-01  3.91030610e-01 -5.52405939e-02
 -2.64506519e-01 -1.68704949e-02  1.08179405e-01 -1.59265280e-01
  1.76367313e-01  1.52895227e-01  2.39540368e-01  4.60776180e-01
  6.79257438e-02  8.53202268e-02  4.65844601e-01 -2.92748790e-02
  1.39548391e-01 -6.25228435e-02 -8.81098136e-02  1.29076734e-01
 -2.87963688e-01 -2.78228223e-01  1.14980549e-01  2.96932131e-01
 -6.15359321e-02 -2.38211

In [20]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('arco', 1.0),
 ('carcaj', 0.7306239604949951),
 ('cinturón', 0.5926663875579834),
 ('flechas', 0.5925911664962769),
 ('cuchillo', 0.5764560103416443),
 ('posición', 0.5206767320632935),
 ('juego', 0.5116968154907227),
 ('lanzo', 0.5080978274345398),
 ('cojo', 0.5058712363243103),
 ('hombro', 0.5021395683288574)]

In [46]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["arco"], topn=10)

[('carcaj', 0.7306239604949951),
 ('cinturón', 0.5926663875579834),
 ('flechas', 0.5925911664962769),
 ('cuchillo', 0.5764560699462891),
 ('posición', 0.5206767320632935),
 ('juego', 0.5116968154907227),
 ('lanzo', 0.5080978274345398),
 ('cojo', 0.5058712363243103),
 ('hombro', 0.5021395683288574),
 ('hoja', 0.5017749667167664)]

In [47]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["pareja"], topn=10)

[('real', 0.6856131553649902),
 ('vencedores', 0.6634766459465027),
 ('seguridad', 0.6529455184936523),
 ('alambrada', 0.6507955193519592),
 ('oportunidad', 0.6478244662284851),
 ('imagen', 0.6424364447593689),
 ('vencedor', 0.6397624015808105),
 ('4', 0.6386364102363586),
 ('única', 0.6359946727752686),
 ('palabra', 0.6337614059448242)]

### 5 - Visualizar agrupación de vectores

In [22]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [23]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [24]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [25]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.




El algoritmo demostró ser eficaz en la tarea de procesar el libro "Los Juegos del Hambre", dividiéndolo en oraciones, tokenizando las palabras y creando embeddings utilizando el modelo Skipgram.

Se pudieron identificar relaciones coherentes entre palabras que se utilizan en conjunto con frecuencia en el contexto de la trama, como:

"Tributo" y "Sobrevivir", lo que refleja el objetivo central de los tributos en los Juegos del Hambre.

"Distrito" y "Arco", que hace referencia a los diferentes distritos y a la habilidad del personaje principal con el arco y las flechas.

A pesar de la eficacia general del algoritmo se observaron limitaciones notables en la identificación de ciertas relaciones clave en el texto. Un ejemplo notable es la falta de una similitud aparente entre "Katniss" y "Peeta", quienes son personajes principales y aliados a lo largo de la trama.

Estas limitaciones sugieren que el modelo de embeddings, en su configuración actual, podría no ser capaz de capturar relaciones más sutiles o específicas entre palabras, incluso cuando estas relaciones son fundamentales en la historia.